<a href="https://colab.research.google.com/github/Nithish271/Colab-notebooks/blob/main/Copy_of_3_3_4_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
l0 = Dense(units=1, input_shape=[1])
model = Sequential([l0])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict([10.0]))
print("Here is what I learned: {}".format(l0.get_weights()))

Epoch 1/500
1/1 [==============================] - 1s 937ms/step - loss: 3.2784
Epoch 2/500
1/1 [==============================] - 0s 23ms/step - loss: 2.7381
Epoch 3/500
1/1 [==============================] - 0s 25ms/step - loss: 2.3097
Epoch 4/500
1/1 [==============================] - 0s 22ms/step - loss: 1.9694
Epoch 5/500
1/1 [==============================] - 0s 10ms/step - loss: 1.6986
Epoch 6/500
1/1 [==============================] - 0s 15ms/step - loss: 1.4825
Epoch 7/500
1/1 [==============================] - 0s 27ms/step - loss: 1.3095
Epoch 8/500
1/1 [==============================] - 0s 10ms/step - loss: 1.1704
Epoch 9/500
1/1 [==============================] - 0s 10ms/step - loss: 1.0581
Epoch 10/500
1/1 [==============================] - 0s 11ms/step - loss: 0.9669
Epoch 11/500
1/1 [==============================] - 0s 11ms/step - loss: 0.8924
Epoch 12/500
1/1 [==============================] - 0s 19ms/step - loss: 0.8311
Epoch 13/500
1/1 [==============================

In [ ]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/1/assets


In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [ ]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

888

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_dense_input:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[10.]]
[[18.98507]]
